In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import plotly.express as px
import pandas as pd
from IPython.display import display
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, median_absolute_error, mean_squared_error,accuracy_score
from xgboost import XGBRegressor

In [ ]:
df_train = pd.read_csv('../input/lish-moa/train_features.csv')
df_test = pd.read_csv('../input/lish-moa/test_features.csv')
df_train_scored = pd.read_csv('../input/lish-moa/train_targets_scored.csv')
df_train_unscored = pd.read_csv('../input/lish-moa/train_targets_nonscored.csv')



### Data Description

In this competition, you will be predicting multiple targets of the Mechanism of Action (MoA) response(s) of different samples (sig_id), given various inputs such as gene expression data and cell viability data.

Two notes:

The training data has an additional (optional) set of MoA labels that are not included in the test data and not used for scoring.
The re-run dataset has approximately 4x the number of examples seen in the Public test.

### Files

**train_features.csv** - Features for the training set. Features g- signify gene expression data, and c- signify cell viability data. cp_type indicates samples treated with a compound (cp_vehicle) or with a control perturbation (ctrl_vehicle); control perturbations have no MoAs; cp_time and cp_dose indicate treatment duration (24, 48, 72 hours) and dose (high or low).
train_targets_scored.csv - The binary MoA targets that are scored.
train_targets_nonscored.csv - Additional (optional) binary MoA responses for the training data. These are not predicted nor scored.
test_features.csv - Features for the test data. You must predict the probability of each scored MoA for each row in the test data.
sample_submission.csv - A submission file in the correct format.



In [ ]:
df_train[:6]

In [ ]:
df_train['cp_type'].value_counts()

In [ ]:
df_train_scored[:5]

In [ ]:
df_train_unscored[:6]

## Pandas Profiling:
### The display.max_columns option controls the number of columns to be printed.It receives an int or None (to print all the columns):

In [ ]:
from pandas_profiling import ProfileReport

In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:
df_train.head(5)

### Profile Report for Features g- which signify gene expression data

In [ ]:
ProfileReport(df_train.iloc[:,4:776],minimal =True)

#### G-307, 3200 distinct counts, 8383 are Zeros (35.2 % of Zeros) which has 13 % of unique values.

#### G-707, 4730 distinct counts.
#### G- 307, is the highest(35.2 % of zeros) and  G-707 is the 2nd highest(31% of zeros) 


### Profile Report for Features c- signify cell viability data

In [ ]:
ProfileReport(df_train.iloc[:,776:],minimal =True)

### Obervation: Minimum for c- signify cell viability feature is -10

In [ ]:

df_train.cp_type.value_counts(normalize=True)

In [ ]:
df_train.cp_time.value_counts(normalize=True)

In [ ]:
df_train.cp_dose.value_counts(normalize=True)

In [ ]:
#### Target values distribution¶


### Target values distribution¶


In [ ]:
plt.hist(df_train_scored.mean())
plt.title('Distribution of mean target in each target column');

In [ ]:
df_train_scored.describe()

In [ ]:
df_train_scored.mean().min(),df_train_scored.mean().max(),df_train_scored.mean().mean()
#We have a high imbalance as  the max target rate is 0.03, the min is very low

In [ ]:
df_train[:2]

In [ ]:
plt.plot(df_train.loc[df_train['sig_id'] == 'id_000644bb2',[col for col in df_train if 'g-' in col]].values.reshape(-1,1));
plt.title('g- value of id_000644bb2');

In [ ]:
plt.plot(sorted(df_train.loc[df_train['sig_id'] == 'id_000644bb2', [col for col in df_train if 'g-' in col]].values.reshape(-1, 1)))
plt.title('sorted g- value of id_000644bb2');

###
Features

    sig_id is the unique sample id
    Features with g- prefix are gene expression features and there are 772 of them (from g-0 to g-771)
    Features with c- prefix are cell viability features and there are 100 of them (from c-0 to g-99)
    cp_type is a binary categorical feature which indicates the samples are treated with a compound or with a control perturbation (trt_cp or ctl_vehicle)
    cp_time is a categorical feature which indicates the treatment duration (24, 48 or 72 hours)
    cp_dose is a binary categorical feature which indicates the dose is low or high (D1 or D2)



In [ ]:
#Sample
sample = pd.read_csv("../input/lish-moa/sample_submission.csv")

#Test
test_features = pd.read_csv("../input/lish-moa/test_features.csv",index_col='sig_id')

#Train
train_features = pd.read_csv("../input/lish-moa/train_features.csv",index_col='sig_id')
train_nonscore = pd.read_csv("../input/lish-moa/train_targets_nonscored.csv",index_col='sig_id')
train_score = pd.read_csv("../input/lish-moa/train_targets_scored.csv",index_col='sig_id')

In [ ]:
g_features = [feature for feature in train_features.columns if feature.startswith('g-')]
c_features = [feature for feature in train_features.columns if feature.startswith('c-')]
other_features = [feature for feature in train_features.columns if feature not in g_features and feature not in c_features]
                                                            

print(f'Number of g- Features: {len(g_features)}')
print(f'Number of c- Features: {len(c_features)}')
print(f'Number of other Features: {len(other_features)} ({other_features})')

In [ ]:
cols = train_score.columns
submission = pd.DataFrame({'sig_id': test_features.index})
total_loss = 0

SEED = 42

In [ ]:
y = train_score[column]
print(y)

In [ ]:
for c, column in enumerate(cols,1):
    
    y = train_score[column]
    
    # Split
    X_train, X_valid, y_train, y_valid = train_test_split(train_features, y, train_size=0.9, test_size=0.1, random_state=SEED)
   
    X_test = test_features.copy()

    # One-hot encoding
    X_train = pd.get_dummies(X_train)
    X_test = pd.get_dummies(X_test)
    X_valid = pd.get_dummies(X_valid)
    
    X_train, X_test = X_train.align(X_test, join='left', axis=1)
    X_train, X_valid = X_train.align(X_valid, join='left', axis=1)
    
    model = XGBRegressor(
                         tree_method = 'gpu_hist',
                         min_child_weight = 31.580,
                         learning_rate = 0.055,
                         colsample_bytree = 0.655,
                         gamma = 3.705,
                         max_delta_step = 2.080,
                         max_depth = 25,
                         n_estimators = 170,
                         #subsample =  0.864, 
                         subsample =  0.910,
                         booster='dart',
                         validate_parameters = True,
                         grow_policy = 'depthwise',
                         predictor = 'gpu_predictor'
                              
                        )
                        
    # Train Model
    model.fit(X_train, y_train)
    pred = model.predict(X_valid)
    
    
    # Loss
    mae = mean_absolute_error(y_valid,pred)
    mdae = median_absolute_error(y_valid,pred)
    mse = mean_squared_error(y_valid,pred)
    
    total_loss += mae
    
    # Prediction
    predictions = model.predict(X_test)
    submission[column] = predictions
    
    print("Regressing through col-"+str(c)+", Mean Abs Error: "+str(mae)+", Median Abs Error: "+str(mdae)+", Mean Sqrd Error: "+str(mse))
    
    

In [ ]:
submission.to_csv('submission.csv', index=False)

In [ ]:
print("Loss: ", total_loss/206)